First, import needed modules

In [25]:
import numpy as np
import pandas as pd
import os

Initalize variables

In [26]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [54]:
controls_raw = pd.read_csv(f'{path}/data/controls/inkar_21_clean.csv', encoding='UTF-8-SIG')

Rename columns

In [55]:
col_names = ['bereich', 'id', 'indikator', 'raumbezug', 'AGS', 'kennziffer_eu', 'name', 'year', 'wert']
controls_raw.columns = col_names
controls_raw.columns

Index(['bereich', 'id', 'indikator', 'raumbezug', 'AGS', 'kennziffer_eu',
       'name', 'year', 'wert'],
      dtype='object')

Fix the AGS

In [56]:
controls_raw['AGS'] = controls_raw['AGS'].astype(str)
controls_raw['AGS'] = np.where(controls_raw['AGS'].str.len() < 8, '0' + controls_raw['AGS'], controls_raw['AGS'])

Convert Wert to numeric

In [57]:
controls_raw['value'] = controls_raw['wert'].str.replace(',','.').astype(float)
controls_raw.dtypes

bereich           object
id                 int64
indikator         object
raumbezug         object
AGS               object
kennziffer_eu    float64
name              object
year               int64
wert              object
value            float64
dtype: object

Split into Municipality and Mun. Association dfs

In [58]:
controls_mun_raw = controls_raw[controls_raw['raumbezug'] == 'Gemeinden']
controls_assoc_raw = controls_raw[controls_raw['raumbezug'] == 'Gemeindeverbände']

Pivot df

In [59]:
controls_mun = pd.pivot(controls_mun_raw, index=['AGS', 'name', 'year'], columns='indikator', values='value')
controls_mun = controls_mun.reset_index()

In [60]:
controls_assoc = pd.pivot(controls_assoc_raw, index=['AGS', 'name', 'year'], columns='indikator', values='wert')
controls_assoc = controls_assoc.reset_index()

In [62]:
controls_assoc['year'].value_counts()

1995    4618
2008    4618
2018    4618
2017    4618
2016    4618
2015    4618
2014    4618
2013    4618
2012    4618
2011    4618
2010    4618
2009    4618
2007    4618
1996    4618
2006    4618
2005    4618
2004    4618
2003    4618
2002    4618
2001    4618
2000    4618
1999    4618
1998    4618
1997    4618
2019    4618
Name: year, dtype: int64

Rename columns and compute statistics

In [52]:
controls_mun['students'] = controls_mun['Studierende'] + controls_mun['Studierende an FH']

In [53]:
col_names = ['AGS', 'name', 'year', 'Bevölkerung', 'Bodenfläche gesamt', 'pop_density', 'Studierende', 'Studierende an FH', 'students']
controls_mun.columns = col_names
controls_mun.columns

Index(['AGS', 'name', 'year', 'Bevölkerung', 'Bodenfläche gesamt',
       'pop_density', 'Studierende', 'Studierende an FH', 'students'],
      dtype='object')

In [ ]:
controls['pop_density'] = controls['Bevölkerung'] / controls['Area']
controls['male'] = controls['männlich'] / controls['Bevölkerung']
controls['female'] = controls['weiblich'] / controls['Bevölkerung']
controls['foreign'] = controls['Ausländer/-innen'] / controls['Bevölkerung']
controls['employed'] = controls['Beschäftigte'] / controls['Bevölkerung']
controls['unemployed'] = controls['Arbeitslose'] / controls['Bevölkerung']
controls['protestant'] = controls['Evangelisch'] / controls['Bevölkerung']
controls['catholic'] = controls['Katholisch'] / controls['Bevölkerung']
controls['religion_other'] = controls['Religion Sonstige/Keine'] / controls['Bevölkerung']
controls = controls[['AGS', 'pop_density', 'female', 'foreign', 'unemployed', 'avg_income', 'avg_age', 'catholic']]
controls.head()

,AGS,pop_density,female,foreign,unemployed,avg_income,avg_age,catholic
0,01000000,177.227657,0.514134,0.041805,0.035730,32200.0,44.5,0.059842
1,01001000,1449.735636,0.507233,0.062353,0.062328,27000.0,43.0,0.063459
2,01002000,1987.206068,0.518602,0.070281,0.053999,27500.0,41.8,0.073661
3,01003000,981.770225,0.525299,0.062562,0.052348,27600.0,44.9,0.084606
4,01004000,1078.444786,0.511838,0.060337,0.056648,26200.0,44.4,0.064972


Export dataset

In [ ]:
controls = controls.set_index('AGS')
controls.to_csv(f'{path}/data/controls.csv', encoding = 'utf-8-sig')
controls.head()

,pop_density,female,foreign,unemployed,avg_income,avg_age,catholic
AGS,,,,,,,
01000000,177.227657,0.514134,0.041805,0.035730,32200.0,44.5,0.059842
01001000,1449.735636,0.507233,0.062353,0.062328,27000.0,43.0,0.063459
01002000,1987.206068,0.518602,0.070281,0.053999,27500.0,41.8,0.073661
01003000,981.770225,0.525299,0.062562,0.052348,27600.0,44.9,0.084606
01004000,1078.444786,0.511838,0.060337,0.056648,26200.0,44.4,0.064972
